# Meta Model 

# x 입력

p_Mag_Width_Ratio
i_Split_Ratio
i_Slot_Depth_Ratio

sampleSize = 20
dimension = 3

# Test Motorcad 호출

self.distribution = ot.ComposedDistribution(
    [self.E, self.F, self.L, self.I], self.copula)

X_train= myDistribution.getSample(sampleSize_train)

Y=IPM_MCAD(X_train)


## Import Package

In [1]:
from os.path import join
from numpy import pi


from pyleecan.Classes.OPdq import OPdq
from pyleecan.definitions import DATA_DIR

from pyleecan.Classes.LamSlotMag import LamSlotMag
from pyleecan.Classes.SlotM11 import SlotM11
from pyleecan.Classes.MachineSIPMSM import MachineSIPMSM
from pyleecan.Classes.Simu1 import Simu1
#from pyleecan.Classes.MagMotorCAD import MagMotorCAD
from pyleecan.Classes.InputVoltage import InputVoltage
from pyleecan.Classes.Output import Output
from pyleecan.Functions.load import load

In [2]:
import win32com.client
import os
import numpy as np
import time
from os import getcwd
from os.path import join, dirname, exists
from math import pi, sqrt
from scipy.io import loadmat

In [3]:
refdir='D:\KDH\Thesis\Rotem_HDEV\MCAD'
wdir = join('D', 'test_MotorCAD_run')
print(refdir)
print(wdir)

D:\KDH\Thesis\Rotem_HDEV\MCAD
D\test_MotorCAD_run


## Class Definition and Method Open_MotorCAD

In [24]:
filename='D:\KDH\Thesis\Rotem_HDEV\MCAD\IPM_Tuto.mot'


filename_=filename
#class Mag(object):
class variables_MotorCAD:
    def __init__(self):
        self.value=[]
        self.name='init_magmotorcad'
        self.o_WLTP3_Eff=0
        self.i_Active_Length     = 110.   # Active length
        self.i_Mag_Thick         = 5.5    # Magnet thickness
        self.i_Mag_Post          = 2.0    # Magnet post
        self.i_Bridge_Thick      = 1.0    # Bridge thickness
        self.i_Pole_V_Angle      = 110.   # V-angle in mech. degrees
        self.i_Split_Ratio       = 0.6    # Ratio of stator bore
        self.i_Tooth_Width_Ratio = 0.5    # Ratio of tooth width
        self.i_Slot_Depth_Ratio  = 0.6    # Ratio of slot depth
        self.i_Slot_Op_Ratio     = 0.7    # Ratio of slot opening
        self.i_Pole_Arc_Ratio    = 0.8    # Ratio of pole arc or magnet V width

        ### Geometry
        self.p_Pole_Pair            = 8.    # Number of rotor poles
        self.p_Stator_Slots         = 24.   # Number of stator slots
        self.p_Tooth_Tip_Depth      = 1.    # Tooth tip depth
        self.p_Tooth_Tip_Angle      = 30.   # Tooth tip angle
        self.p_Stator_OD            = 300.  # Stator outer diameter
        self.p_EndSpace_Height      = 15.   # Space between winding ends and caps
        self.p_Wdg_Overhang         = 20.   # Winding overhang height
        self.p_Airgap_Mecha         = 1.    # Mechanical airgap
        self.p_Mag_Clear            = 0.5   # Magnet clearance
        self.p_Shaft_OD_Ratio       = 0.7   # Ratio of shaft diameter
        self.p_Mag_Width_Ratio      = 1.0   # Ratio of magnet width
        self.p_Mag_Separation_Min   = 2.0    # Minimum separation between magnet poles
        self.p_Shaft_Separation_Min = 5.0   # Minimum separation between shaft and magnets
        self.p_Mag_AspectRatio_Min  = 0.5   # Minimum magnet aspect ratio

        ### Winding
        self.p_Coils_Slot    = 2.    # Number of coils going through each slot
        self.p_Turns_Coil    = 40.   # Number of turns per coil
        self.p_Parallel_Path = 8.    # Number of parallel paths per phase
        self.p_Slot_Fill     = 0.4   # Copper slot fill factor

        ### Materials
        self.p_Yield_Rotor  = 460.   # Rotor core yield strength
        self.p_Temp_Wdg_Max = 180.   # Maximum winding temperature
        self.p_Temp_Mag_Max = 140.   # Maximum magnet temperature

        ### Performance
        self.p_Speed_Max        = 7000.   # Maximum operating speed
        self.p_Line_Current_RMS = 500.    # Maximum RMS line current

        ### Calculation settings
        self.p_Speed_Lab_Step    = 100.                              # Speed step used in Lab
        self.p_Speed_Peak_Array  = np.array([500., 3000., 6000.])    # Speeds for peak performance calculation 
        self.p_Speed_Cont_Array  = np.array([1000., 5000.])          # Speeds for continuous performance calculation
        self.p_Torque_Pts        = 90                                # Timesteps per cycle for torque calculation                                              

        ### Post-processing
        Pic_Export = 1      # Export geometry snapshots (0: No; 1: Yes)

        ### Dependent parameters
        self.Speed_Max_Rad = pi*self.p_Speed_Max/30                                               # Maximum speed in radians
        self.Speed_Lab     = np.arange(0, self.p_Speed_Max +self.p_Speed_Lab_Step, self.p_Speed_Lab_Step)  # Speed vector in Lab
        self.Speed_Lab     = self.Speed_Lab.tolist()                                              # Required for signal generation
        self.Speed_Lab_Len = len(self.Speed_Lab)                

        ### Geometry parameters
        self.Machine_Length = fun_Machine_Length(self.p_EndSpace_Height, self.p_Wdg_Overhang, self.i_Active_Length)
        self.Air_Pocket     = fun_Air_Pocket(self.i_Mag_Thick, self.p_Mag_Clear)
        self.Active_Volume  = fun_Active_Volume(self.p_Stator_OD, self.i_Active_Length)     # In [m3]
        
        # Required for signal generation

        
    

In [9]:
# Machine length
def fun_Machine_Length(p_EndSpace_Height, p_Wdg_Overhang, i_Active_Length):
    res = i_Active_Length + 2*(p_EndSpace_Height + p_Wdg_Overhang)
    return res

# Active volume
def fun_Active_Volume(Stator_OD, Active_Length):  
    res = pi*Stator_OD**2/4*Active_Length*1e-9
    return res

# Air pocket
def fun_Air_Pocket(Mag_Thick, Mag_Clear):
    res = (Mag_Thick + Mag_Clear)/2 
    return res

### Performances

# Torque density
def fun_Torque_Density(Torque, Volume):           
    res = Torque/Volume*1e-3
    return res
 
# Stress safety factor
def fun_Stress_Safety(Rotor_Yield, Stress_Max):            
    res = Rotor_Yield/Stress_Max
    return res

In [10]:
        
class Mag:
    def __init__(self):
        self.name='init_magmotorcad'
        self.o_WLTP3_Eff=0
        self.i_Active_Length     = 110.   # Active length
        self.i_Mag_Thick         = 5.5    # Magnet thickness
        self.i_Mag_Post          = 2.0    # Magnet post
        self.i_Bridge_Thick      = 1.0    # Bridge thickness
        self.i_Pole_V_Angle      = 110.   # V-angle in mech. degrees
        self.i_Split_Ratio       = 0.6    # Ratio of stator bore
        self.i_Tooth_Width_Ratio = 0.5    # Ratio of tooth width
        self.i_Slot_Depth_Ratio  = 0.6    # Ratio of slot depth
        self.i_Slot_Op_Ratio     = 0.7    # Ratio of slot opening
        self.i_Pole_Arc_Ratio    = 0.8    # Ratio of pole arc or magnet V width

        ### Geometry
        self.p_Pole_Pair            = 8.    # Number of rotor poles
        self.p_Stator_Slots         = 24.   # Number of stator slots
        self.p_Tooth_Tip_Depth      = 1.    # Tooth tip depth
        self.p_Tooth_Tip_Angle      = 30.   # Tooth tip angle
        self.p_Stator_OD            = 300.  # Stator outer diameter
        self.p_EndSpace_Height      = 15.   # Space between winding ends and caps
        self.p_Wdg_Overhang         = 20.   # Winding overhang height
        self.p_Airgap_Mecha         = 1.    # Mechanical airgap
        self.p_Mag_Clear            = 0.5   # Magnet clearance
        self.p_Shaft_OD_Ratio       = 0.7   # Ratio of shaft diameter
        self.p_Mag_Width_Ratio      = 1.0   # Ratio of magnet width
        self.p_Mag_Separation_Min   = 2.0    # Minimum separation between magnet poles
        self.p_Shaft_Separation_Min = 5.0   # Minimum separation between shaft and magnets
        self.p_Mag_AspectRatio_Min  = 0.5   # Minimum magnet aspect ratio

        ### Winding
        self.p_Coils_Slot    = 2.    # Number of coils going through each slot
        self.p_Turns_Coil    = 40.   # Number of turns per coil
        self.p_Parallel_Path = 8.    # Number of parallel paths per phase
        self.p_Slot_Fill     = 0.4   # Copper slot fill factor

        ### Materials
        self.p_Yield_Rotor  = 460.   # Rotor core yield strength
        self.p_Temp_Wdg_Max = 180.   # Maximum winding temperature
        self.p_Temp_Mag_Max = 140.   # Maximum magnet temperature

        ### Performance
        self.p_Speed_Max        = 7000.   # Maximum operating speed
        self.p_Line_Current_RMS = 500.    # Maximum RMS line current

        ### Calculation settings
        self.p_Speed_Lab_Step    = 100.                              # Speed step used in Lab
        self.p_Speed_Peak_Array  = np.array([500., 3000., 6000.])    # Speeds for peak performance calculation 
        self.p_Speed_Cont_Array  = np.array([1000., 5000.])          # Speeds for continuous performance calculation
        self.p_Torque_Pts        = 90                                # Timesteps per cycle for torque calculation                                              

        ### Post-processing
        Pic_Export = 1      # Export geometry snapshots (0: No; 1: Yes)

        ### Dependent parameters
        self.Speed_Max_Rad = pi*self.p_Speed_Max/30                                               # Maximum speed in radians
        self.Speed_Lab     = np.arange(0, self.p_Speed_Max +self.p_Speed_Lab_Step, self.p_Speed_Lab_Step)  # Speed vector in Lab
        self.Speed_Lab     = self.Speed_Lab.tolist()                                              # Required for signal generation
        self.Speed_Lab_Len = len(self.Speed_Lab)                

        ### Geometry parameters
        self.Machine_Length = fun_Machine_Length(self.p_EndSpace_Height, self.p_Wdg_Overhang, self.i_Active_Length)
        self.Air_Pocket     = fun_Air_Pocket(self.i_Mag_Thick, self.p_Mag_Clear)
        self.Active_Volume  = fun_Active_Volume(self.p_Stator_OD, self.i_Active_Length)     # In [m3]
        
        # Required for signal generation
                    
   

    def Open_MotorCAD(self, filename):
        Visible_Opt     = 1.           # Set Motor-CAD visible
        Message_Display = 2.           # Display all pop-up messages 
        Save_Prompt     = 1.           # Never prompt to save file
       
        
        if filename is not None:
            # Open MagMotorCAD instance if filename is not None (parallel case)
            try:
        # Try to open MagMotorCAD instance if handler already exists (first parallelized handler)
                mcApp=win32com.client.Dispatch("MotorCAD.AppAutomation")
                mcApp.Visible = Visible_Opt
                # mcApp.LoadFromFile(mot_file_ref_path)    # Load reference Motor-CAD file
                mcApp.SetVariable('MessageDisplayState', Message_Display)   # Set state of message display 
                mcApp.SetVariable("SavePrompt", Save_Prompt)    
                mcApp.LoadFromFile(filename)  
                print(filename)
            except Exception:
                print('File is not exist Check Filename')
            #     # Create a new FEMM handler in case of parallelization on another FEMM instance
            #     mcApp=win32com.client.Dispatch("MotorCAD.AppAutomation") 
            #     print(filename)
            # Open FEMM file
        else:
            # FEMM instance and file is already open, get filename from output
            print('already openend')
            # mcApp.LoadFromFile(mot_file_ref_path)    # Load reference Motor-CAD file

    
    
    def Parameter_Change_MotorCAD(self, x):
        mcApp=win32com.client.Dispatch("MotorCAD.AppAutomation")
        mcApp.SetVariable('GeometryParameterisation', 1)

        #Check initial Value
        a=mcApp.GetVariable("MagnetThickness_Array")
        # a=mcApp.GetVariable("VSimpleMagnetPost_Array") 
        print(a)

        #Change Value
        i_Mag_Thick         = x   # Magnet thickness
        mcApp.SetArrayVariable("MagnetThickness_Array", 0, self.i_Mag_Thick) 
        b=mcApp.GetVariable("MagnetThickness_Array")
        print(b)

        if a==b:
            print("Already changed")
        else:
            print("Changed")
    
    def Solve_Lab(self):
        p_Speed_Max = 7000. 
        p_Line_Current_RMS =500. 
        p_Speed_Lab_Step    = 100. 
    
        ## Build Lab Model
        mcApp.ClearModelBuild_Lab()  # Clear existing models
        mcApp.SetVariable("ModelType_MotorLAB", 2)       # Saturation model type: Full Cycle
        mcApp.SetVariable("SatModelPoints_MotorLAB", 0)  # Saturation model: coarse resolution (15 points)    
        mcApp.SetVariable("LossModel_LAB", 0)            # Loss model type: neglect
        mcApp.SetMotorLABContext()                       # Lab context
        mcApp.SetVariable("BuildSatModel_MotorLAB", 1)   # Activate saturation model   
    
        mcApp.SetVariable("MaxModelCurrent_RMS_MotorLAB", p_Line_Current_RMS)       # Max line current (rms)
        mcApp.SetVariable("MaxModelCurrent_MotorLAB", p_Line_Current_RMS*sqrt(2))   # Max line current (peak)
        mcApp.SetVariable('ModelBuildSpeed_MotorLAB', p_Speed_Max)                  # Maximum operating speed
        mcApp.BuildModel_Lab()             
        
        ## Lab: peak performance
        mcApp.SetVariable("OperatingMode_Lab", 0)                      # Motoring mode
        mcApp.SetVariable("EmagneticCalcType_Lab", 0)                  # Peak performance
        mcApp.SetVariable('SpeedMax_MotorLAB', p_Speed_Max)            # Maximum speed
        mcApp.SetVariable('Speedinc_MotorLAB', p_Speed_Lab_Step)       # Speed step
        mcApp.SetVariable('Imax_RMS_MotorLAB', p_Line_Current_RMS)     # Max line current (rms)
        mcApp.SetVariable('Imax_MotorLAB', p_Line_Current_RMS*sqrt(2)) # Max line current (peak) 
        mcApp.CalculateMagnetic_Lab()    

        ### Lab: efficiency over WLTP-3 drive cycle
        mcApp.SetVariable("DutyCycleType_Lab", 1)                 # Automotive drive cycle
        mcApp.SetVariable("DrivCycle_MotorLAB", "WLTP Class 3")   # WLTP3 drive cycle
        mcApp.SetVariable("LabThermalCoupling_DutyCycle", 0)      # No coupling with Thermal
        mcApp.CalculateDutyCycle_Lab() 
    
    def Output_to_dict(self):
        ex,o_WLTP3_Eff = mcApp.GetVariable("DutyCycleAverageEfficiency_EnergyUse")   # Get efficiency value 
        return self.o_WLTP3_Eff


In [ ]:
test_mag.Parameter_Change_MotorCAD(6)
test_mag.Solve_Lab()
test_mag.Output_to_dict()


In [11]:
test_mag=Mag()

In [ ]:
test_mag.Open_MotorCAD(filename_)


In [ ]:
test_mag.Parameter_Change_MotorCAD(6)

In [ ]:
test_mag.Solve_Lab()


In [ ]:
test_mag.Output_to_dict()

In [ ]:
mcApp=win32com.client.Dispatch("MotorCAD.AppAutomation")
ex,o_WLTP3_Eff = mcApp.GetVariable("DutyCycleAverageEfficiency_EnergyUse")   # Get efficiency value 


# Not sureParameter_Change_MotorCAD

In [ ]:

mcApp=win32com.client.Dispatch("MotorCAD.AppAutomation")
mcApp.SetVariable('GeometryParameterisation', 1)

#Check initial Value
a=mcApp.GetVariable("MagnetThickness_Array")
# a=mcApp.GetVariable("VSimpleMagnetPost_Array") 
print(a)

#Change Value
i_Mag_Thick         = 5.5    # Magnet thickness
mcApp.SetArrayVariable("MagnetThickness_Array", 0, i_Mag_Thick) 
b=mcApp.GetVariable("MagnetThickness_Array")
print(b)

if a==b:
    print("Already changed")
else:
    print("Changed")
    


# Calculation & Data management


In [ ]:
                             # Run calculation
Mat_File_Name     = 'MotorLAB_elecdata.mat'                   # *.mat file automatically generated by Motor-CAD
Mat_File_Path     = join(mot_file_dir, 'Lab', Mat_File_Name)  # Point to the *.mat file
Mat_File_Data     = loadmat(Mat_File_Path)                    # Load data from the *.mat file
Mat_File_Speed    = Mat_File_Data['Speed']                    # Load speed data
Mat_File_Torque   = Mat_File_Data['Shaft_Torque']             # Load shaft torque data    
Mat_File_Power    = Mat_File_Data['Shaft_Power']              # Load shaft power data
Mat_File_Torque   = Mat_File_Torque.flatten()                 # Necessary to be read by list_2_list_variant()
Mat_File_Power    = Mat_File_Power.flatten()  
Mat_File_Speed    = Mat_File_Speed.flatten()

In [ ]:
Design_Name     = "IPM_tuto" 
refdir

Machine_Length = fun_Machine_Length(p_EndSpace_Height, p_Wdg_Overhang, i_Active_Length)
Air_Pocket     = fun_Air_Pocket(i_Mag_Thick, p_Mag_Clear)
Active_Volume  = fun_Active_Volume(p_Stator_OD, i_Active_Length)     # In [m3]

## Make New file for OSL    

mot_file_new_path = join(wdir, Design_Name + '_' + OSL_DESIGN_NAME + ".mot") # Path to the new *.mot file
print("[INFO] Design file saved as: ", mot_file_new_path)                                    
mcApp.SaveToFile(mot_file_new_path)                                          # Save in new location
mot_file_dir = join(wdir, Design_Name + '_' + OSL_DESIGN_NAME)
mcApp.SetVariable('GeometryParameterisation', 1)   # Ratio mode in Motor-CAD 

                             
                    

In [ ]:
machine_1 = load(join(DATA_DIR, "Machine", "Toyota_Prius.json"))


In [ ]:
class Test_MagMotorCAD(object):
    def Test_MagMotorCAD_LAB(self):
        """Test StructElmer simulation with 2 magnets on HoleM50 rotor"""

        # copy the machine
        machine = machine_1.copy()


        # setup the simulation
        ##
        simu = Simu1(name="test_StructElmer_HoleM50", machine=machine)
        ##
        output = Output(simu=simu)
        output.path_result = save_path

        ## simu1의 입력이 Magmotorcad파일로 갔다고 쳐야됨
        
        
        simu.mag=MagMotorCAD()
        
        # set rotor speed and run simulation
        simu.run()

        return output

    def __init__(
        self,
        name="",
        desc="",
        machine=-1,
        input=-1,
        logger_name="Pyleecan.Simulation",
        var_simu=None,
        postproc_list=-1,
        index=None,
        path_result=None,
        layer=None,
        layer_log_warn=None,
        init_dict=None,
        init_str=None,
    ):
        """Constructor of the class. Can be use in three ways :
        - __init__ (arg1 = 1, arg3 = 5) every parameters have name and default values
            for pyleecan type, -1 will call the default constructor
        - __init__ (init_dict = d) d must be a dictionary with property names as keys
        - __init__ (init_str = s) s must be a string
        s is the file path to load

        ndarray or list can be given for Vector and Matrix
        object or dict can be given for pyleecan Object"""

# !!! On goging Optimization 정의


In [12]:
dir(test_mag)

['Active_Volume',
 'Air_Pocket',
 'Machine_Length',
 'Open_MotorCAD',
 'Output_to_dict',
 'Parameter_Change_MotorCAD',
 'Solve_Lab',
 'Speed_Lab',
 'Speed_Lab_Len',
 'Speed_Max_Rad',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'i_Active_Length',
 'i_Bridge_Thick',
 'i_Mag_Post',
 'i_Mag_Thick',
 'i_Pole_Arc_Ratio',
 'i_Pole_V_Angle',
 'i_Slot_Depth_Ratio',
 'i_Slot_Op_Ratio',
 'i_Split_Ratio',
 'i_Tooth_Width_Ratio',
 'name',
 'o_WLTP3_Eff',
 'p_Airgap_Mecha',
 'p_Coils_Slot',
 'p_EndSpace_Height',
 'p_Line_Current_RMS',
 'p_Mag_AspectRatio_Min',
 'p_Mag_Clear',
 'p_Mag_Separation_Min',
 'p_Mag_Width_Ratio',
 'p_Parallel_Path',
 'p_Pole_Pair',
 'p_Shaft_OD_Ratio',


In [13]:
test_ref=test_mag

## !!!! On going Design variables

In [19]:
test_ref.p_Tooth_Tip_Depth

1.0

In [40]:
vari_mag=variables_MotorCAD()

In [48]:
def gen_setter(i):
    def new_setter(vari_mag, value):
        vari_mag.value = value

    return new_setter

In [49]:
my_vars = []
for i in range(2):
    my_vars.append(
        OptiDesignVar(
            name="Ir({})".format(i),
            symbol="var_" + str(i),
            type_var="interval",
            space=[0, 1],
            get_value=lambda space: np.random.uniform(*space),
            setter=gen_setter(i),
        )
    )


In [50]:
a= my_vars[1].get_value(my_vars[1].space)
print(a)
my_vars[1]
vari_mag.value

0.3531864576860947


[]

In [22]:
from pyleecan.Classes.OptiDesignVar import OptiDesignVar
import random
# Design variables

my_design_var=[]
def gen_setter(self,i):
    def new_setter(self, value):
        self.p_Tooth_Tip_Depth[i] = value

    return new_setter

for i in range(30):
    my_design_var.append( 
        OptiDesignVar(
            name="p_Tooth_Tip_Depth",
            symbol = "ttd",
            unit = "mm",
            type_var="interval",
            space=[
                0 * test_ref.p_Tooth_Tip_Depth,
                test_ref.p_Tooth_Tip_Depth,
            ],
            get_value="lambda space: random.uniform(*space)", # To initiate randomly the first generation
            setter="gen_setter(i)", # Variable to edit
        )
    )

CheckTypeError: For property setter Expected function or str (path to python file or lambda), got: <class 'str'>

## OptiObjective
## Datakeeper??

In [ ]:
from pyleecan.Classes.OptiObjective import OptiObjective
import numpy as np



def harm1(output):
    """Return the first torque harmonic """
    harm_list = output.mag.Tem.get_magnitude_along("freqs")["T_{em}"]
    
    # Return the first torque harmonic
    return harm_list[1] 

my_obj = [
    OptiObjective(
        name="Maximization of the average torque",
        symbol="Tem_av",
        unit="N.m",
        keeper="lambda output: -abs(output.mag.Tem_av)",
    ),
    OptiObjective(
        name="Minimization of the first torque harmonic",
        symbol="Tem_h1",
        unit="N.m",
        keeper=harm1,
    ),
]

# run

In [ ]:
Y = g(X)

# Train

# Validation

# Optimization